#  Tuning parameters for RandomForest model

In [1]:
import numpy as np
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## 1. Load data

In [2]:
import pandas as pd

data_train = pd.read_csv('../../data/interim/trainFinal.csv')
data_test = pd.read_csv('../../data/interim/testFinal.csv')
data_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_code
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING,2
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING,2
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING,2
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING,2
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING,2


## 2. Create X and y variables

In [3]:
X_train = data_train.drop(['Activity', 'Activity_code'], axis=1)
y_train = np.array(data_train['Activity_code'])

X_test = data_test.drop(['Activity', 'Activity_code'], axis=1),
y_test = data_test['Activity_code']

## 3. GridSearchCV for RandomForest

In [4]:
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.model_selection import GridSearchCV

# Khởi tạo mô hình
model = RandomForestClassifier()

# Lưới siêu tham số
param_grid = {
    'max_depth': [None, 20, 30],             # Độ sâu tối đa của cây
    'min_samples_split': [2, 5, 10],             # Số mẫu tối thiểu để chia nhánh
    'min_samples_leaf': [1, 2, 5],               # Số mẫu tối thiểu tại một node lá
    'bootstrap': [True, False],                  # Có sử dụng bootstrap hay không
}

# Khởi tạo GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,                    # Số lần cross-validation
    scoring='accuracy',      # Metric đánh giá
    verbose=1,               # Hiển thị chi tiết hơn tiến trình
    n_jobs=-1                # Sử dụng toàn bộ CPU
)

# Theo dõi thời gian
start_time = time.time()

# Thực hiện tìm kiếm
print("Starting GridSearchCV...")
grid_search.fit(X_train, y_train)

# Tính toán thời gian
elapsed_time = time.time() - start_time

# Kết quả
print("GridSearchCV completed!")
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print('Best params: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)


Starting GridSearchCV...
Fitting 3 folds for each of 54 candidates, totalling 162 fits
GridSearchCV completed!
Elapsed time: 127.93 seconds
Best params:  {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best score:  0.9182538294795682


## 4. Save the best model

In [8]:
import os
import json

# Dữ liệu để lưu vào file JSON
data = {
    "bootstrap": grid_search.best_params_['bootstrap'],
    "max_depth": grid_search.best_params_['max_depth'],
    "min_samples_leaf": grid_search.best_params_['min_samples_leaf'],
    "min_samples_split": grid_search.best_params_['min_samples_split']
}

# Đường dẫn và tên file JSON
directory = "Best_Hyperparameter/"
filename = "best_parameter_RandomForest.json"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(directory, exist_ok=True)

# Đường dẫn đầy đủ của file JSON
file_path = os.path.join(directory, filename)

# Ghi dữ liệu vào file JSON
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

print(f"File JSON đã được tạo tại: {file_path}")


File JSON đã được tạo tại: Best_Hyperparameter/best_parameter_RandomForest.json


## 5. Results

##### Random Forest
* GridSearchCV completed!
* Elapsed time: 247.81 seconds
* Best params:  {'C': 16, 'coef0': 0, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'}
* Best score:  0.9446426142876572